#Analyzing the NYC Subway Dataset

This project consists of two parts. In Part 1 of the project, you should have completed the questions in Problem Sets 2, 3, and 4 in the Introduction to Data Science course.
This document addresses part 2 of the project. Please use this document as a template and answer the following questions to explain your reasoning and conclusion behind your work in the problem sets. You will attach a document with your answers to these questions as part of your final project submission.

## <font color='red'>The Data I used:</font>

<font color='red'>- The improved data provided by : https://www.dropbox.com/s/meyki2wl9xfa7yk/turnstile_data_master_with_weather.csv and  https://www.dropbox.com/s/1lpoeh2w6px4diu/improved-dataset.zip?dl=0 
<br/>

Since the data used in Problem Sets in the course "Intro to Data Science" is incomplete, I don't know whether the data subsets chosen by the course is the same at each problem. So I use both the "Original data set" and "Improved data set" provided by the links above and do all the analysis in this project on my own computer.</font>

### 0. References

https://www.dropbox.com/s/1lpoeh2w6px4diu/improved-dataset.zip?dl=0 
https://s3.amazonaws.com/uploads.hipchat.com/23756/665149/05bgLZqSsMycnkg/turnstile-weather-variables.pdf
http://stackoverflow.com/questions/19377371/how-to-make-a-histogram-in-ipython-notebook-using-ggplot2-for-python


In [39]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats

path = r'~/Documents/GitHub/DataAnalystNanodegree/improved-dataset/turnstile_weather_v2.csv'

turnstile_weather = pd.read_csv(path)

enter_rain = turnstile_weather[turnstile_weather.rain == 1]['ENTRIESn_hourly'].reset_index()
enter_not_rain = turnstile_weather[turnstile_weather.rain == 0]['ENTRIESn_hourly'].reset_index()

### 1. Statistical Test

#### 1.1 Which statistical test did you use to analyze the NYC subway data? Did you use a one-tail or a two-tail P value? What is the null hypothesis? What is your p-critical value?

1. Mann Whitney U-test
2. two-tail P value 
3. null hypothesis : the population mean for riderships are the same no matter the weather is rainy or not at that day
4. p-critical : 0.05 （0.025 for one tailed result)

#### 1.2 Why is this statistical test applicable to the dataset? In particular, consider the assumptions that the test is making about the distribution of ridership in the two samples.

Since the distribution for riderships is not a normal distribution, I can't use normal T-test to analyze the data, so I need to use non-paramatric tests, that is, the Mann Whitney U-test.

In [4]:
w1,p1 = scipy.stats.shapiro(enter_rain['ENTRIESn_hourly'])
w2,p2 = scipy.stats.shapiro(enter_not_rain['ENTRIESn_hourly'])

/Users/ArthurlLin/anaconda3/lib/python3.4/site-packages/scipy/stats/morestats.py:997: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")


#### 1.3 What results did you get from this statistical test? These should include the following numerical values: p-values, as well as the means for each of the two samples under test.

1. original data set
    1. mean of riderships on rainy day: 1105.45
    2. mean of riderships on non-rainy day: 1090.28
    3. p-value = 0.02499991

2. improved data set
    1. mean of riderships on rainy day: 2028.20
    2. mean of riderships on non-rainy day: 1845.54
    3. p-value = 2.74e-06

#### 1.4 What is the significance and interpretation of these results?

1. original data set:
    0.02499991 < 0.025, The result is significance, We reject the Null Hypothesis
    
2. improved data set:
    2.74e-06 < 0.025, The result is significance, We reject the Null Hypothesis

### 2. Linear Regression 

#### 2.1. What approach did you use to compute the coefficients theta and produce prediction for ENTRIESn_hourly in your regression model:

B. Gradient descent using Scikit Learn

#### 2.2 What features (input variables) did you use in your model? Did you use any dummy variables as part of your features?

'rain', 'precipi', 'meantempi', 'Hour', 'day_week' and 'UNIT' as dummy variable

#### 2.3 Why did you select these features in your model? We are looking for specific reasons that lead you to believe that

1. I don't think fog, wind and pressure will affect people's intention to ride subway or not much
2. I want to use thunder column, but the value is zero for all the data, so scipy didn't allow me to put it into features ( and would be useless if it can) .
3. I think time is a very important feature, since must people will take subway regularly at particular days and time, so I use 'hour' and create a column called 'day_week' from 'DATEn' to see which day it is in a week.
4. I put 'UNIT' as dummy variable because my R^2 is hugely imporved by it


#### 2.4 What are the parameters (also known as "coefficients" or "weights") of the non-dummy features in your linear regression model?

1. original data set : -1.74828282e+02, 3.44697319e+01, -5.35755971e+00, 6.24658642e+01, -6.79772926e+01 <br/>
2. improved data set : 101.21725504, -3428.09409717, -15.98932694, 124.4798835, -142.04545176

#### 2.5 What is your model’s R2 (coefficients of determination) value

1. original data set : 0.442139378169 <br/>
2. improved data set : 0.461523395628

#### 2.6 What does this R2 value mean for the goodness of fit for your regression model? Do you think this linear model to predict ridership is appropriate for this dataset, given this R2  value?

1. This means the data is not very close to the regression line, the prediction power of this model is poor. 
2. No!

### 3. Visualization

#### 3.1 One visualization should contain two histograms: one of  ENTRIESn_hourly for rainy days and one of ENTRIESn_hourly for non-rainy days.

In [18]:
from ggplot import *
enter_rain = enter_rain[enter_rain.ENTRIESn_hourly < 20000]
df = turnstile_weather[['rain','ENTRIESn_hourly']]
df = df[df.ENTRIESn_hourly < 15000]
df['rain'] = df['rain'] == 1

p = ggplot(aes(x='ENTRIESn_hourly',fill='rain'), data=df) + ggtitle( 'Frequency By ENTRIESn_hourly' ) + xlab('ENTRIESn_hourly') + ylab('Frequency')
p + geom_histogram(binwidth=1000)

/Users/ArthurlLin/anaconda3/lib/python3.4/site-packages/ggplot/geoms/geom_bar.py:47: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  _reset = self.bottom == None or (self.ax != None and self.ax != ax)


<ggplot: (279988726)>

#### 3.2 One visualization can be more freeform. You should feel free to implement something that we discussed in class (e.g., scatter plots, line plots) or attempt to implement something more advanced if you'd like. 

In [38]:
entries_per_hour = turnstile_weather.groupby('hour').sum().reset_index()
p = ggplot( entries_per_hour, aes( 'hour', 'ENTRIESn_hourly') ) + ggtitle( 'hour By ENTRIESn_hourly' ) + geom_point( color = "red" ) + geom_line( color = "red")
print(p)

<ggplot: (-9223372036575008694)>


In [37]:
turnstile_weather['day_week'] = pd.to_datetime(turnstile_weather['DATEn']).dt.weekday
entries_per_weekday = turnstile_weather.groupby('day_week').sum().reset_index()
p = ggplot( entries_per_weekday, aes( 'day_week', 'ENTRIESn_hourly') ) + ggtitle( 'day_week By ENTRIESn_hourly' ) + geom_point( color = "red" ) + geom_line( color = "red")
print(p)

<ggplot: (-9223372036575020919)>


### 4.Conclusion

#### 4.1 From your analysis and interpretation of the data, do more people ride the NYC subway when it is raining or when it is not raining?  

#### 4.2 What analyses lead you to this conclusion? You should use results from both your statistical tests and your linear regression to support your analysis.

1. original data set:
    - From the result of Mann Whitney U-test, we can see the riderships on rainy days is more than those on non-rainy days, but the difference is very small, and the p value is barely inside the 'signification zone' for p-critical at 0.05. However, from the result of linear regression, the 'weight' of rain is negative (-1.49419295e+02), that means there are negative correlation between rain and ridership. So if I need to make a conclusion by these results, I would say that there are more people ride the NYC subway when it is not raining.
    
2. improved data set:
    - The result generated from the improved data is totally different. First of all, the result of Mann Whitney U-test is very significant, and the 'weight' of parameter 'rain' comes from linear regression is positive, I would say that there are more people ride the NYC subway when it is raining. 

3. Conclusion:
    - The result from the improved data match my intuition better that raining will let people want to ride subway since they can shelter from rain while inside the train. However, the R^2 value for each data set is small, I think the weight parameter comes from each result doesn't have much meaning. If we only take the result comes from Mann Whitney U-test into consideration, the result, that the more people will ride subway when it is raining, still match my intuition.

### 5. Reflection

#### 5.1 Please discuss potential shortcomings of the methods of your analysis, including:

1. Although using the UNIT as dummy variable improve my R^2 value hugely, it's wierd to take each UNIT as a feature to the regression model. I don't know if putting a variable with so many different types as dummy variable is a good way to do the linear regression? 


2. The result comes from the improved data seems better than the original data, but I don't know what is the criteria for choosing data, maybe the result can be improved if I can clean off more useless data.

3. Linear model may not be the best one to predict the ridership.


#### 5.2 (Optional) Do you have any other insight about the dataset that you would like to share with us?

I try to randomly split ten percent of data as testing data and use the other as traing data, and calculate the R^2 value using the testing data and the model generated by the training data, the R^2 value doesn't change too much, So I think the model doesn't overfit, but I still feel uncomfortable using UNIT as dummy variable. The source code I used to make this experiment is shown below.

In [1]:
def predictions_by_test_data(dataframe):

    dataframe['day_week'] = pandas.to_datetime(dataframe['DATEn']).dt.weekday

    #Separate Data
    rows = random.sample( list(dataframe.index), round(len(dataframe.index)/10) )
    df_test = dataframe.ix[rows]
    df_train = dataframe.drop(rows)

    #Choose features
    features_test = df_test[['rain', 'precipi', 'meantempi', 'hour', 'day_week']]
    dummy_units = pandas.get_dummies(df_test['UNIT'], prefix='unit')
    features_test = features_test.join(dummy_units)

    features_train = df_train[['rain', 'precipi', 'meantempi', 'hour', 'day_week']]
    dummy_units = pandas.get_dummies(df_train['UNIT'], prefix='unit')
    features_train = features_train.join(dummy_units)

    # Values
    values_test = df_test['ENTRIESn_hourly']
    values_train = df_train['ENTRIESn_hourly']

    # Get numpy arrays
    features_array = features_train.values
    values_array = values_train.values
    means, std_devs, normalized_features_array = normalize_features(features_array)

    # Perform gradient descent
    norm_intercept, norm_params = linear_regression(normalized_features_array, values_array)
    intercept, params = recover_params(means, std_devs, norm_intercept, norm_params)

    predictions = intercept + np.dot(features_test.values, params)
    print(getRsquare(predictions, values_test))


def getRsquare(predictions, data):

    numerator = 0
    denominator = 0
    data_mean = data.mean()

    print(data_mean)

    for idx, val in enumerate(data):
        numerator += np.square(val - predictions[idx])
        denominator += np.square(val - data_mean)

    r_squared = 1 - numerator/denominator
    return r_squared